In [343]:
import json
from base64 import b64decode
import f90nml
from collections import OrderedDict
import os
import html
from textwrap import indent

In [344]:
def generate_tag_webpages(database, root):
    for nl in database.keys():
        for tag in database[nl].keys():
            v = database[nl][tag]
            if v["html"] != "":
                webpage = b64decode(v["html"]).decode("utf-8")
                path = os.path.join(root, f'{tag}.html')
                with open(path, "w") as f:
                    f.write(webpage)

In [345]:
def nl_print(string):
    if string is True:
        return '.true.'
    elif string is False:
        return '.false.'
    elif isinstance(string, str):
        return f"'{string}'"
    else:
        return string

def find_comment_indents(namelists):
    line_len = {}
    for nl in namelists.keys():
        for tag, val in namelists[nl].items():
            if isinstance(val, list):
                this_tag_len = []
                for i, v in enumerate(val):
                    # TODO: this is off by one space usually...
                    string = f"{tag}({i+1}) = {nl_print(val)}"
                    this_tag_len.append(len(string))
                line_len.update({tag: this_tag_len})
            else:
                string = f"{tag} = {nl_print(val)}"
                line_len.update({tag: len(string)})

    # Find max length of all the tags, accounting for lists
    full_lens = []
    for v in line_len.values():
        if isinstance(v, list):
            full_lens.extend(v)
        else:
            full_lens.append(v)
    max_len = max(full_lens)

    indents = {}
    for k, v in line_len.items():
        if isinstance(v, list):
            indents[k] = [(max_len - i)*' ' for i in v]
        else:
            indents[k] = (max_len - v)*' '

    return indents

def generate_tutorial(filename, database, root):
    ind=2 # Indentation level
    input_file = f90nml.read(filename)
    with open(filename, 'r') as f:
        cards = f.read().split('/')[-1]
    
    # Maxmimum length of a namelist line
    comment_indents = find_comment_indents(input_file)

    base_link = "file:///" + os.path.join(root, "INPUT_PW.html")
    tag_html = '<div class="row">\n<div class="column left">\n<div class="input-file">\n'
    preview_html = '<div class="cloumn right">\n'
    for nl in input_file.keys():
        tag_html += indent(html.escape(f"&{nl}\n"), ' '*ind)
        namelist = input_file[nl]
        for tag, val in namelist.items():
            idm = database[nl][tag]['idm']
            link = base_link+"#"+idm
            link = f'<a href="{link}" class = "tag-link" id="{tag}">{tag}</a>'

            if database[nl][tag]['options']:
                options = database[nl][tag]['options']
                comment = options.get(str(val), None)
                if comment is None:
                    comment = "Unknown Value"
            elif database[nl][tag]['info']:
                comment = database[nl][tag]['info']
            else:
                comment = ''
            comment = ' ! ' + comment if comment else ''
            comment = comment.split('.')[0]
            comment = comment.split('-')[0]
            comment = comment.split('(')[0]
            comment = comment.split('see')[0]
            comment = comment.split(':')[0]

            tag_link = ''
            if isinstance(val, list):
                for i, v in enumerate(val):
                    cind = comment_indents[tag][i]
                    print(f'For tag({i+1}) {tag}, val is {v}, cind is {cind.replace(" ", "*")}')
                    tag_link += f"{link}({i+1}) = {nl_print(v)}{cind}{comment}\n"
            else:
                cind = comment_indents[tag]
                print(f'For tag {tag}, val is {val}, cind is {cind.replace(" ", "*")}')
                tag_link += f"{link} = {nl_print(val)}{cind}{comment}\n"
            tag_html += indent(tag_link, '  '*ind)

            preview_link = "file:///" + os.path.join(root, "tags", f"{tag}.html")
            preview = f'<div class="preview" id="preview_{tag}">\n'
            preview += f'<object data="{preview_link}" class="preview-object" type="text/html">'
            preview += ' </object>\n</div>\n'
            preview_html += indent(preview, ' '*ind)

        tag_html += f"{' '*ind}/\n"
    preview_html += "</div>"
    tag_html += cards
    tag_html += "</div>\n</div>\n"
    webpage_html = tag_html + preview_html

    # TODO: use local jquery
    webpage = f"""<!DOCTYPE html>
    <html>
    <head>
    {' '*ind}<link rel="stylesheet" type="text/css" href="style.css"></link>
    {' '*ind}<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.6.4/jquery.min.js"></script>
    </head>
    <body>
    <h2>Input File</h2>
    The following information was parsed from <tt>{filename}</tt>:

    {webpage_html}
    <script src="script.js"></script>
    </body>
    </html>
    """
    path = os.path.join(root, 'tutorial.html')
    with open(path, 'w') as f:
        f.write(webpage)

In [346]:
filename = 'pw.in'
database_dir = f'database/'
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist

filename = os.path.abspath(filename)
database_dir = os.path.abspath(database_dir)
root = os.getcwd()
working_dir = os.path.join(root, '.dft-tutor')
database_filename = os.path.join(database_dir, 'qe-'+version, 'database.json')


# Load Database
with open(database_filename) as f:
    database = json.load(f)
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
tags_dir = os.path.join(working_dir, 'tags')
if not os.path.exists(tags_dir):
    os.mkdir(tags_dir)
generate_tag_webpages(database, tags_dir)
generate_tutorial(filename, database, working_dir)
import shutil
style_css = os.path.join(database_dir, 'style.css')
tags_css = os.path.join(database_dir, 'qe-tag.css')
script_js = os.path.join(database_dir, 'script.js')
docs_html = os.path.join(database_dir, 'qe-'+version, 'INPUT_PW.html')
shutil.copy2(style_css, working_dir)
shutil.copy2(tags_css, working_dir)
shutil.copy2(docs_html, working_dir)
shutil.copy2(script_js, working_dir)

For tag calculation, val is bands, cind is ***
For tag etot_conv_thr, val is 2e-05, cind is ***
For tag forc_conv_thr, val is 0.0001, cind is **
For tag outdir, val is ./out/, cind is *******
For tag prefix, val is si, cind is ***********
For tag pseudo_dir, val is ./pseudo/, cind is 
For tag tprnfor, val is True, cind is ********
For tag tstress, val is True, cind is ********
For tag verbosity, val is high, cind is ******
For tag ecutrho, val is 240.0, cind is *********
For tag ecutwfc, val is 30.0, cind is **********
For tag ibrav, val is 2, cind is ***************
For tag(1) celldm, val is 10.26, cind is *****
For tag nat, val is 2, cind is *****************
For tag nosym, val is False, cind is *********
For tag ntyp, val is 1, cind is ****************
For tag occupations, val is fixed, cind is ***
For tag nbnd, val is 8, cind is ****************
For tag conv_thr, val is 4e-10, cind is ********
For tag electron_maxstep, val is 80, cind is ***
For tag mixing_beta, val is 0.4, cind is

'/Users/ashour/code/dft-tutor/.dft-tutor/script.js'

In [347]:
namelists = f90nml.read(filename)